<a href="https://colab.research.google.com/github/E-tech-coder/DataScienceCapstoneProject/blob/Elena/RuleBasedDepartment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
!git clone https://github.com/E-tech-coder/DataScienceCapstoneProject.git

fatal: destination path 'DataScienceCapstoneProject' already exists and is not an empty directory.


# Simple Bag of Words

In [250]:
import pandas as pd

df = pd.read_csv("/content/DataScienceCapstoneProject/linkedin_experience_annotated.csv").drop(columns="linkedin").set_index("person_id").reset_index()

In [251]:
df.head(20)

,person_id,organization,position,startDate,endDate,status,department,seniority
0,0,Depot4Design GmbH,Prokurist,2019-08,NaN,ACTIVE,Other,Management
1,0,Depot4Design GmbH,CFO,2019-07,NaN,ACTIVE,Other,Management
2,0,Depot4Design GmbH,Betriebswirtin,2019-07,NaN,ACTIVE,Other,Professional
3,0,Depot4Design GmbH,Prokuristin,2019-07,NaN,ACTIVE,Other,Management
4,0,Depot4Design GmbH,CFO,2019-07,NaN,ACTIVE,Other,Management
5,0,Nagel Car Group,Buchhalterin,2000-05,2019-06,INACTIVE,Other,Professional
6,1,Computer Solutions,Solutions Architect,2024-03,NaN,ACTIVE,Information Technology,Professional
7,1,Computer Solutions,Senior Network Engineer,2019-07,2024-03,INACTIVE,Information Technology,Senior
8,1,Texas A&M University-Corpus Christi,Manager of Network Services,2017-02,2019-07,INACTIVE,Information Technology,Professional
9,1,Texas A&M University-Corpus Christi,Infrastructure Administrator II,2015-06,2017-02,INACTIVE,Information Technology,Professional


In [252]:
dpt = pd.read_csv("/content/DataScienceCapstoneProject/department-v2.csv")

In [253]:
dpt['label'].drop_duplicates()

,label
0,Marketing
1,Project Management
171,Administrative
202,Business Development
374,Consulting
476,Human Resources
487,Information Technology
2609,Other
2613,Purchasing
2636,Sales


In [254]:
import re

BOW = dict()

for label in dpt['label'].drop_duplicates() :
  x = []
  for text in dpt[dpt["label"] == label]["text"]:
    tokens = re.split(r"[ ./&-()|-]", text.lower().strip())
    tokens = [t for t in tokens if t]
    x.extend(tokens)
  BOW[label] = list(set(x))


In [255]:
label

'Customer Support'

In [256]:
df["WordAppearance"] = 0
df["PredictedPosition_BOW"] =""


In [257]:
import re

def split_position(text):
  # add an empty space before capital letters
  text = re.sub(r'(?<!^)(?=[A-Z])', " ", text)

  # Lowercase and split on space, /, &, -
  return re.split(r'[ /&-]+',text.lower().strip())

In [258]:
df["position_words"] = df["position"].apply(split_position)

In [259]:
df["position_words"]

,position_words
0,[prokurist]
1,"[c, f, o]"
2,[betriebswirtin]
3,[prokuristin]
4,"[c, f, o]"
...,...
2633,[justitiar]
2634,[geschäftsführer]
2635,[präsidium]
2636,[rechtsanwalt]


In [260]:
for label, BOW_words in BOW.items():
  bow_set = set(BOW_words)
  # Count how many words of the position appear in the bag of words of the label
  for idx,words in df["position_words"].items():
    count=sum(word in bow_set for word in words)
        # Update prediction if the current word count is greater than the previous one
    if count > df.at[idx, "WordAppearance"] :
      df.at[idx, "WordAppearance"] = count
      df.at[idx, "PredictedPosition_BOW"] = label

In [261]:
df.to_csv("/content/DataScienceCapstoneProject/2BOW-Position-Predict.csv")

# TF-IDF with Sklearn

In [262]:
# The performance using only bag of words for classification is very terrible.
# Let's try TFIDF method.

In [263]:
corpus = []
labels = []

for key, value in BOW.items():
  labels.append(key)
  corpus.append(value)


In [264]:
len(corpus)

11

In [265]:
from sklearn.feature_extraction.text import TfidfVectorizer
# turn seperated words into a string sentence in each corpus
corpus = [" ".join(doc) for doc in corpus]
titles = labels

vectorizer = TfidfVectorizer(ngram_range=(1,1), lowercase=True)
vector = vectorizer.fit_transform(corpus)

In [266]:
dict(zip(vectorizer.get_feature_names_out(), vector.toarray()[0]))

{'10': np.float64(0.0),
 '1st': np.float64(0.0),
 '2010': np.float64(0.027526015958889743),
 '2020': np.float64(0.0),
 '2h': np.float64(0.027526015958889743),
 '365': np.float64(0.01849162972807137),
 '3d': np.float64(0.027526015958889743),
 '3rd': np.float64(0.0),
 '42he': np.float64(0.0),
 '4hana': np.float64(0.0),
 '8man': np.float64(0.027526015958889743),
 'ab': np.float64(0.0),
 'abm': np.float64(0.027526015958889743),
 'about': np.float64(0.0),
 'abrasives': np.float64(0.0),
 'abt': np.float64(0.0),
 'abteilung': np.float64(0.01849162972807137),
 'abteilungsdirektor': np.float64(0.0),
 'abteilungsleiter': np.float64(0.01849162972807137),
 'abteilungsleiterin': np.float64(0.020691766603149888),
 'abteilungsleitung': np.float64(0.023528236365204208),
 'abteilungsverantwortlicher': np.float64(0.027526015958889743),
 'academia': np.float64(0.027526015958889743),
 'academy': np.float64(0.027526015958889743),
 'acccount': np.float64(0.0),
 'accelerating': np.float64(0.0),
 'accelerator

In [267]:
tfidf_df = pd.DataFrame(vector.toarray(), index = titles, columns = vectorizer.get_feature_names_out())

In [268]:
tfidf_df.shape

(11, 3405)

In [269]:
tfidf_df.columns

Index(['10', '1st', '2010', '2020', '2h', '365', '3d', '3rd', '42he', '4hana',
       ...
       'öffentliche', 'öffentlichk', 'öffentlichke', 'öffentlichkeit',
       'öffentlichkeitsarbeit', 'öffentlichkeitsmarketing', 'öpnv',
       'österreich', 'özkazanc', '高级营销专家'],
      dtype='object', length=3405)

In [270]:
# for label, BOW_words in BOW.items():
#   bow_set = set(BOW_words)
#   # Count how many words of the position appear in the bag of words of the label
#   for idx,words in df["position_words"].items():
#     count=sum(word in bow_set for word in words)
#         # Update prediction if the current word count is greater than the previous one
#     if count > df.at[idx, "WordAppearance"] :
#       df.at[idx, "WordAppearance"] = count
#       df.at[idx, "PredictedPosition_BOW"] = label

In [271]:
df["Tfidf_score"] = 0
df["tfidf_predict"] = " "

In [272]:
for idx, words in df["position_words"].items():
  for label in tfidf_df.index :
    score = 0
    for word in words :
      if word in list(tfidf_df.columns) and (tfidf_df.at[label,word] != 0) :
        score = score + tfidf_df.at[label,word]
    if score > df.at[idx, "Tfidf_score"]:
      df.at[idx, "Tfidf_score"] = score
      df.at[idx, "tfidf_predict"] = label


/tmp/ipython-input-2041765854.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.013857517247410026' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, "Tfidf_score"] = score


In [273]:
df.to_csv("/content/DataScienceCapstoneProject/TFIDF-Position-Predict.csv")

# Cosine Similarity

In [274]:
df

,person_id,organization,position,startDate,endDate,status,department,seniority,WordAppearance,PredictedPosition_BOW,position_words,Tfidf_score,tfidf_predict
0,0,Depot4Design GmbH,Prokurist,2019-08,NaN,ACTIVE,Other,Management,1,Marketing,[prokurist],0.101070,Purchasing
1,0,Depot4Design GmbH,CFO,2019-07,NaN,ACTIVE,Other,Management,2,Marketing,"[c, f, o]",0.000000,
2,0,Depot4Design GmbH,Betriebswirtin,2019-07,NaN,ACTIVE,Other,Professional,1,Marketing,[betriebswirtin],0.027526,Marketing
3,0,Depot4Design GmbH,Prokuristin,2019-07,NaN,ACTIVE,Other,Management,1,Marketing,[prokuristin],0.130022,Other
4,0,Depot4Design GmbH,CFO,2019-07,NaN,ACTIVE,Other,Management,2,Marketing,"[c, f, o]",0.000000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633,607,Bistum,Justitiar,2014-03,NaN,ACTIVE,Other,Professional,0,,[justitiar],0.000000,
2634,607,FORESTA Management,Geschäftsführer,2010-01,NaN,ACTIVE,Other,Management,1,Marketing,[geschäftsführer],0.118184,Other
2635,607,Malteser Hilfsdienst,Präsidium,1999,NaN,ACTIVE,Other,Management,0,,[präsidium],0.000000,
2636,607,Spies & Brunner RAe,Rechtsanwalt,1995-01,NaN,ACTIVE,Other,Professional,0,,[rechtsanwalt],0.000000,


In [275]:
df["position_words"].items()

In [276]:
"management" in tfidf_df.columns

True

In [277]:
#####################################################################################

In [278]:
# Cosine similarity

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# turn seperated words into a string sentence in each corpus
# corpus = [" ".join(doc) for doc in corpus]
# titles = labels

# vectorizer = TfidfVectorizer(ngram_range=(1,1))
# vector = vectorizer.fit_transform(corpus)
# Since we have fit the vectorize before, we only need to transform it.

X_positions = vectorizer.transform(df['position'])
similarity = cosine_similarity(X_positions, tfidf_df)

import numpy as np
df["cosine_predict"] = tfidf_df.index[similarity.argmax(axis = 1)]
df["cosine_ifidf_score"] = similarity.max(axis = 1)




In [279]:
df.to_csv("/content/DataScienceCapstoneProject/Cosine-Position-Predict.csv")